<a href="https://colab.research.google.com/github/archiesboy/predictive_maint/blob/main/pred_maint_mct_ict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#ml-based predictive maintenance using sensor data
#group CKTDU ict423

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("ai4i2020.csv")


In [ ]:
data.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0


In [ ]:
data.shape

(10000, 14)

In [ ]:
data.columns

Index(['UDI', 'Product ID', 'Type', 'Air temperature [K]',
       'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]',
       'Tool wear [min]', 'Machine failure', 'TWF', 'HDF', 'PWF', 'OSF',
       'RNF'],
      dtype='object')

In [ ]:
data.isnull().sum()

,0
UDI,0
Product ID,0
Type,0
Air temperature [K],0
Process temperature [K],0
Rotational speed [rpm],0
Torque [Nm],0
Tool wear [min],0
Machine failure,0
TWF,0


In [ ]:
data['Machine failure'].value_counts()

,count
Machine failure,
0,9661
1,339


In [ ]:
cleandata = data.drop( columns = ['UDI', 'Product ID', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF' ])

In [ ]:
cleandata.head()

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure
0,M,298.1,308.6,1551,42.8,0,0
1,L,298.2,308.7,1408,46.3,3,0
2,L,298.1,308.5,1498,49.4,5,0
3,L,298.2,308.6,1433,39.5,7,0
4,L,298.2,308.7,1408,40.0,9,0


In [ ]:
encodeddata = pd.get_dummies(cleandata, columns = ['Type'], drop_first= True)

In [ ]:
encodeddata.head()

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,Type_L,Type_M
0,298.1,308.6,1551,42.8,0,0,False,True
1,298.2,308.7,1408,46.3,3,0,True,False
2,298.1,308.5,1498,49.4,5,0,True,False
3,298.2,308.6,1433,39.5,7,0,True,False
4,298.2,308.7,1408,40.0,9,0,True,False


In [ ]:
x= encodeddata.drop('Machine failure', axis=1)
y= encodeddata['Machine failure']

In [ ]:
x.head(), y.head()

(   Air temperature [K]  Process temperature [K]  Rotational speed [rpm]  \
 0                298.1                    308.6                    1551   
 1                298.2                    308.7                    1408   
 2                298.1                    308.5                    1498   
 3                298.2                    308.6                    1433   
 4                298.2                    308.7                    1408   
 
    Torque [Nm]  Tool wear [min]  Type_L  Type_M  
 0         42.8                0   False    True  
 1         46.3                3    True   False  
 2         49.4                5    True   False  
 3         39.5                7    True   False  
 4         40.0                9    True   False  ,
 0    0
 1    0
 2    0
 3    0
 4    0
 Name: Machine failure, dtype: int64)

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
from numpy import random
trainx, testx, trainy, testy = train_test_split(
    x, y,
    test_size = 0.2,
    random_state= 1,
    stratify =y
)

In [ ]:
trainx.shape, testx.shape

((8000, 7), (2000, 7))

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
scaled_trainx = scaler.fit_transform(trainx)
scaled_testx = scaler.transform(testx)

# Display the first few rows of the scaled training data
import pandas as pd
scaled_trainx_df = pd.DataFrame(scaled_trainx, columns=trainx.columns)
display(scaled_trainx_df.head())

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Type_L,Type_M
0,-0.655536,-0.005128,0.088626,-0.736892,1.395211,0.815221,-0.651343
1,-0.356430,-0.543989,-1.149152,0.634860,-1.207400,-1.226661,1.535289
2,-0.755239,-0.409273,-0.101802,-0.246266,-0.219662,-1.226661,1.535289
3,0.191933,1.005237,0.923145,-1.027263,-0.078557,0.815221,-0.651343
4,1.139105,1.072594,-0.549866,1.395832,-0.533230,-1.226661,-0.651343


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Initialize the Logistic Regression model
model = LogisticRegression(random_state=1)

# Train the model using the scaled training data
model.fit(scaled_trainx, trainy)

# Make predictions on the scaled test data
predictions = model.predict(scaled_testx)

# Evaluate the model
print("Accuracy:", accuracy_score(testy, predictions))
print("\nClassification Report:\n", classification_report(testy, predictions))
print("\nConfusion Matrix:\n", confusion_matrix(testy, predictions))

Accuracy: 0.969

Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98      1932
           1       0.75      0.13      0.23        68

    accuracy                           0.97      2000
   macro avg       0.86      0.57      0.60      2000
weighted avg       0.96      0.97      0.96      2000


Confusion Matrix:
 [[1929    3]
 [  59    9]]


In [ ]:
!pip install imblearn

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
scaled_trainx_resampled, trainy_resampled = smote.fit_resample(scaled_trainx, trainy)

print(f'Original training data shape: {scaled_trainx.shape}')
print(f'Resampled training data shape: {scaled_trainx_resampled.shape}')
print(f'Original training target distribution:\n{trainy.value_counts()}')
print(f'Resampled training target distribution:\n{trainy_resampled.value_counts()}')

Original training data shape: (8000, 7)
Resampled training data shape: (15458, 7)
Original training target distribution:
Machine failure
0    7729
1     271
Name: count, dtype: int64
Resampled training target distribution:
Machine failure
0    7729
1    7729
Name: count, dtype: int64


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Initialize the Random Forest Classifier model
rf_model = RandomForestClassifier(random_state=42)

# Train the model using the SMOTE-resampled training data
rf_model.fit(scaled_trainx_resampled, trainy_resampled)

# Make predictions on the scaled test data (important: use the original test set, not resampled)
rf_predictions = rf_model.predict(scaled_testx)

# Evaluate the Random Forest model
print("Random Forest Model Evaluation:")
print("Accuracy:", accuracy_score(testy, rf_predictions))
print("\nClassification Report:\n", classification_report(testy, rf_predictions))
print("\nConfusion Matrix:\n", confusion_matrix(testy, rf_predictions))

Random Forest Model Evaluation:
Accuracy: 0.964

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.97      0.98      1932
           1       0.48      0.76      0.59        68

    accuracy                           0.96      2000
   macro avg       0.74      0.87      0.79      2000
weighted avg       0.97      0.96      0.97      2000


Confusion Matrix:
 [[1876   56]
 [  16   52]]
